In [164]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.utils import class_weight
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix 
print("TensorFlow version:", tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from datetime import datetime
from sklearn.metrics import accuracy_score
import gc
import warnings
warnings.filterwarnings('ignore')

TensorFlow version: 2.12.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15227683719947374622
xla_global_id: -1
]


In [8]:
RANDOM_SEED = 42

In [9]:
tf.test.is_built_with_cuda()

False

In [10]:
def nan_click_viewed__deleting(X, y = None):
    X = X[X['click'].isna() == False]
    X = X[X['view'] != 0]
    return X 

In [11]:
def dub_dropping(X, y = None):
    X =  X.drop_duplicates()                              
    return X

In [12]:
def nan_filling(X, y = None):
    nan_cols = {}
    for col in X.columns:
        nan_count = X[col].isna().sum()
        if nan_count != 0:
            per = np.round((nan_count / df.shape[0]) * 100, 2)
            nan_cols[col] = per
                       
    for col, per in nan_cols.items():
        if per < 5:
            X = X[X[col].isna() == False]
        else:
            X[col] = X[col].fillna('unknown')
            
    return X

In [100]:
df_clickhouse = pd.read_csv('../internship/clickhouse.csv')
df_clickhouse.shape

(5889167, 39)

In [101]:
df_creatives = pd.read_csv('../internship/creatives.csv')
df_creatives.shape

(132, 15)

In [102]:
df_creatives = df_creatives.rename(columns={'id': 'creative_id'})

In [103]:
df = pd.merge(df_clickhouse, df_creatives, on="creative_id")
df.shape

(5889167, 53)

In [104]:
del df_clickhouse
del df_creatives
gc.collect()

623

In [105]:
df = nan_filling(dub_dropping(nan_click_viewed__deleting(df)))

In [106]:
df_train, df_test = train_test_split(df, stratify = df['click'], shuffle=True, test_size=0.2, random_state=RANDOM_SEED)

X_train = df_train.drop('click', axis = 1)
y_train = df_train['click']

X_test = df_test.drop('click', axis = 1)
y_test = df_test['click']

In [107]:
del df
del df_train
del df_test
gc.collect()

0

In [108]:
def column_deleting(X, y = None):
    columns_for_drop = ['auction_date_time', 'impression_hash', 'ssp', 'is_pay',  'pay_price', 'auction_id', 'impression_id', 'bid_id', 'auction_type', 'bid_floor', 'bid_price', 'is_win', 'view', 'place_number', 'ssp_user_id', 'campaign_id_x', 'enter_utm_campaign',  'stream_id_x', 'link_id', 'format', 'ip_v4', 'ip_v6', 'tag_id', 'iab_category_x', 'ab_test', 'campaign_id_y', 'user_id', 'stream_id_y', 'is_deleted', 'image', 'created_at', 'updated_at']
    X = X.drop(columns_for_drop, axis = 1)
    X.reset_index(drop=True, inplace=True)
    return X

In [109]:
def feature_reduction(X, y = None):
    X['browser'] = X['browser'].apply(lambda x : 'Chrome' if x == 'Google' else x)
    X['browser'] = X['browser'].apply(lambda x : 'Yandex Browser' if x == 'Yandex' else x)
    X.reset_index(drop=True, inplace=True)
    return X

In [110]:
def date_time_col_transformation(X, y = None):
    X['req_date_time'] = pd.to_datetime(X['event_date_time'], format='%Y-%m-%d %H:%M:%S')
    X['day'] =  X['req_date_time'].dt.day
    X['month'] =  X['req_date_time'].dt.month
    X['year'] =  X['req_date_time'].dt.year
    X['hour'] =  X['req_date_time'].dt.hour
    X['day_of_week'] = X['event_date_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").weekday())
    X['is_holyday'] = X['day_of_week'].apply(lambda x : 1 if x in [5 , 6] else 0)
    X['is_friday'] = X['day_of_week'].apply(lambda x : 1 if x == 4 else 0)
    X['is_monday'] = X['day_of_week'].apply(lambda x : 1 if x == 0 else 0)
    X['day_part'] = X['hour'].apply(lambda x : 'Night' if x >= 22 and x < 5 else ('Morning' if x >= 5 and x < 12 else ('Day' if x >= 12 and x < 17 else 'Evening')))
    X['year_part'] = X['month'].apply(lambda x : 'Winter' if x in [12, 1, 2] else ('Spring' if x in [3, 4, 5] else ('Summer' if x in [6, 7, 8] else 'Autumn')))
    X = X.drop(['event_date_time', 'req_date_time', 'year'], axis = 1)
    X.reset_index(drop=True, inplace=True)
    return X

In [111]:
def type_transformation(X, y = None):
    X['loss_reason'] = X['loss_reason'].astype('int8')
    X['device'] = X['device'].astype('int8')
    X['day'] = X['day'].astype('int8')
    X['month'] = X['month'].astype('int8')
    X['hour'] = X['hour'].astype('int8')
    X['day_of_week'] = X['day_of_week'].astype('int8')
    X['is_holyday'] = X['is_holyday'].astype('int8')
    X['is_friday'] = X['is_friday'].astype('int8')
    X['is_monday'] = X['is_monday'].astype('int8')
    X.reset_index(drop=True, inplace=True)
    return X

In [112]:
ohe = OneHotEncoder(sparse = False, max_categories = 500, handle_unknown = 'ignore')
#ohe = OneHotEncoder(sparse = True, handle_unknown = 'ignore')

In [113]:
def cat_encoding(X, y = None):
    cat_cols = X.columns
    ohe.fit(X[cat_cols])
    ft = ohe.transform(X[cat_cols])
    df_ft = pd.DataFrame(ft, columns=ohe.get_feature_names_out())
    X.reset_index(drop=True, inplace=True)
    df_ft.reset_index(drop=True, inplace=True)
    df_ft = df_ft.astype('uint8')
    X = pd.concat([X, df_ft], axis=1)
    X.drop(cat_cols, inplace = True, axis = 1)
    
    return X

In [114]:
preprocessor = Pipeline(
    steps=
    [
        ('column_deleting', FunctionTransformer(func=column_deleting)),
        ('feature_reduction', FunctionTransformer(func=feature_reduction)),
        ('date_time_col_transformation', FunctionTransformer(func=date_time_col_transformation)),
        ('type_transformation', FunctionTransformer(func=type_transformation)),
        ('cat_encoding', FunctionTransformer(func=cat_encoding)),
        
    ]
)

In [115]:
X_train_prep = preprocessor.fit_transform(X_train, y_train)
X_train_prep.shape

(2392377, 929)

In [116]:
X_train_prep.head()

,loss_reason_0,loss_reason_1,loss_reason_2,loss_reason_4,loss_reason_5,loss_reason_6,loss_reason_9,loss_reason_10,loss_reason_11,loss_reason_12,...,is_holyday_0,is_holyday_1,is_friday_0,is_friday_1,is_monday_0,is_monday_1,day_part_Day,day_part_Evening,day_part_Morning,year_part_Winter
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [117]:
X_test_prep = preprocessor.fit_transform(X_test, y_test)
X_test_prep.shape

(598095, 905)

In [118]:
X_test_prep.head()

,loss_reason_0,loss_reason_1,loss_reason_2,loss_reason_4,loss_reason_5,loss_reason_6,loss_reason_9,loss_reason_10,loss_reason_11,loss_reason_12,...,is_holyday_0,is_holyday_1,is_friday_0,is_friday_1,is_monday_0,is_monday_1,day_part_Day,day_part_Evening,day_part_Morning,year_part_Winter
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [150]:
X_train_tensor = tf.convert_to_tensor(X_train_prep)
y_train_tensor = tf.convert_to_tensor(y_train)

X_test_tensor = tf.convert_to_tensor(X_test_prep)
y_test_tensor = tf.convert_to_tensor(y_test)

In [120]:
kf = KFold(n_splits = 5, shuffle = True, random_state = RANDOM_SEED)

In [131]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(100, input_dim = X_train_prep.shape[1], activation='sigmoid'),
  tf.keras.layers.Dense(128, activation='sigmoid'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(len(y_train.unique()))
])

In [159]:
loss_fn = tf.keras.losses.BinaryCrossentropy()

In [162]:
model.compile(optimizer='adam',
              loss=loss_fn,
              #metrics=['accuracy']
              metrics=[tf.keras.metrics.BinaryCrossentropy()]
              #metrics = [tf.keras.metrics.AUC()]
             )

In [163]:
model.fit(X_train_tensor, y_train, epochs=5)

Epoch 1/5
74762/74762 [==============================] - 101s 1ms/step - loss: 0.4246 - binary_crossentropy: 0.4246
Epoch 2/5
74762/74762 [==============================] - 102s 1ms/step - loss: 0.4246 - binary_crossentropy: 0.4246
Epoch 3/5
74762/74762 [==============================] - 102s 1ms/step - loss: 0.4246 - binary_crossentropy: 0.4246
Epoch 4/5
74762/74762 [==============================] - 100s 1ms/step - loss: 0.4246 - binary_crossentropy: 0.4246
Epoch 5/5
74762/74762 [==============================] - 102s 1ms/step - loss: 0.4246 - binary_crossentropy: 0.4246


In [169]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [170]:
predictions = model(X_test_tensor)
predictions

ValueError: Input 0 of layer "sequential_3" is incompatible with the layer: expected shape=(None, 929), found shape=(598095, 905)

In [171]:
probs = probability_model(X_test_tensor)
probs

ValueError: Input 0 of layer "sequential_6" is incompatible with the layer: expected shape=(None, 929), found shape=(598095, 905)

In [ ]:
roc_auc_score(y_test, probs)